In [1]:
from sleepmate.agent import *
from sleepmate.user import *
from sleepmate.diary import *
from sleepmate.isi import *

In [35]:
!rm -f /Users/cck197/Downloads/virtual_greg/memory.pkl

# Database

In [4]:
db_user = User(name="Chris", email="chris@nourishbalancethrive.com")

In [5]:
db_user.save()

<User: User object>

In [2]:
# Querying the User Document
db_user = DBUser.objects(email="chris@nourishbalancethrive.com").first()

In [18]:
db_user = get_current_user()

In [19]:
db_user.to_json()

'{"_id": {"$oid": "6526d86e6015be5b3434ac13"}, "name": "Chris", "email": "chris@nourishbalancethrive.com"}'

# Meet and Greet

In [3]:
memory = X.load_memory()

load_memory filename='/Users/cck197/Downloads/virtual_greg/memory.pkl'


In [9]:
memory.save_context({"input": f"my name is {user.name}"}, {"output": f"hi {user.name}"})

In [5]:
memory.save_context({"input": f"my email is {user.email}"}, {"output": f"got it"})

In [37]:
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='my name is Chris'),
  AIMessage(content='hi Chris'),
  HumanMessage(content='hey'),
  AIMessage(content='Hi Chris! How can I assist you today?'),
  HumanMessage(content='Tell me a joke.'),
  AIMessage(content='\n{"setup": "Why did the chicken cross the road?", "punchline": "To get to the other side!"}'),
  HumanMessage(content="what's my name?"),
  AIMessage(content='\n{"setup": "What\'s my name?", "punchline": ""}'),
  HumanMessage(content="what's my name?"),
  AIMessage(content='\n{"setup": "What\'s my name?", "punchline": ""}'),
  HumanMessage(content="what's my name?"),
  AIMessage(content='\n{"name": "Your Name"}'),
  HumanMessage(content="what's the human's name?"),
  AIMessage(content='\n{"name": "John"}'),
  HumanMessage(content="what's the human's name?"),
  AIMessage(content='\n\n{"name": "Chris"}'),
  HumanMessage(content='my email is chris@nourishbalancethrive.com'),
  AIMessage(content='got it')]}

In [59]:
x = X(goals["diary_probe"], memory=memory);

get_agent len(tools)=10


Hello, Chris! How are you feeling today?

# Parsing

In [5]:
from typing import List

from langchain.llms import OpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain.pydantic_v1 import BaseModel, Field, validator

In [6]:
model_name = "text-davinci-003"
temperature = 0.0
model = OpenAI(model_name=model_name, temperature=temperature)

In [7]:
# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

    # You can add custom validation logic easily with Pydantic.
    @validator("setup")
    def question_ends_with_question_mark(cls, field):
        if field[-1] != "?":
            raise ValueError("Badly formed question!")
        return field


# And a query intented to prompt a language model to populate the data structure.
joke_query = "Tell me a joke."

# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

_input = prompt.format_prompt(query=joke_query)

output = model(_input.to_string())

parser.parse(output)

Joke(setup='Why did the chicken cross the road?', punchline='To get to the other side!')

In [18]:
_input.to_string()

'Answer the user query.\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"setup": {"title": "Setup", "description": "question to set up a joke", "type": "string"}, "punchline": {"title": "Punchline", "description": "answer to resolve the joke", "type": "string"}}, "required": ["setup", "punchline"]}\n```\nTell me a joke.\n'

# Using

In [38]:
# Define your desired data structure.
class UserParser(BaseModel):
    name: str = Field(description="name")
    email: str = Field(description="email address")

In [39]:
# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=UserParser)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\nPrevious conversation:{chat_history}",
    input_variables=["query", "chat_history"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [40]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

In [41]:
chain = LLMChain(
    llm=model,
    prompt=prompt,
    verbose=True,
    memory=memory
)

In [55]:
user_query = "what are the human's details"

In [56]:
output = chain({"query": user_query}); output



> Entering new LLMChain chain...
Prompt after formatting:
Answer the user query.
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"name": {"title": "Name", "description": "name", "type": "string"}, "email": {"title": "Email", "description": "email address", "type": "string"}}, "required": ["name", "email"]}
```
what are the human's details
Previous conversation:[HumanMessage(content='my name is Chris'), AIMessage(content='hi Chris'), HumanMessage(content='hey'), AIMessage(content='Hi Chris! How can I assist you today?'), HumanMessage(content='Tell me a joke.'), AIMessage(co

{'query': "what are the human's details",
 'chat_history': [HumanMessage(content='my name is Chris'),
  AIMessage(content='hi Chris'),
  HumanMessage(content='hey'),
  AIMessage(content='Hi Chris! How can I assist you today?'),
  HumanMessage(content='Tell me a joke.'),
  AIMessage(content='\n{"setup": "Why did the chicken cross the road?", "punchline": "To get to the other side!"}'),
  HumanMessage(content="what's my name?"),
  AIMessage(content='\n{"setup": "What\'s my name?", "punchline": ""}'),
  HumanMessage(content="what's my name?"),
  AIMessage(content='\n{"setup": "What\'s my name?", "punchline": ""}'),
  HumanMessage(content="what's my name?"),
  AIMessage(content='\n{"name": "Your Name"}'),
  HumanMessage(content="what's the human's name?"),
  AIMessage(content='\n{"name": "John"}'),
  HumanMessage(content="what's the human's name?"),
  AIMessage(content='\n\n{"name": "Chris"}'),
  HumanMessage(content='my email is chris@nourishbalancethrive.com'),
  AIMessage(content='got i

In [57]:
parser.parse(output["text"])

UserParser(name='Chris', email='chris@nourishbalancethrive.com')

In [46]:
get_parsed_output(user_query, memory, UserParser)



> Entering new LLMChain chain...
Prompt after formatting:
Answer the user query.
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"name": {"title": "Name", "description": "name", "type": "string"}, "email": {"title": "Email", "description": "email address", "type": "string"}}, "required": ["name", "email"]}
```
what are the human's name and email?
Previous conversation:[HumanMessage(content='my name is Chris'), AIMessage(content='hi Chris'), HumanMessage(content='hey'), AIMessage(content='Hi Chris! How can I assist you today?'), HumanMessage(content='Tell me a joke.'), AIMe

UserParser(name='Chris', email='chris@nourishbalancethrive.com')

In [47]:
up = _

In [49]:
up.dict()

{'name': 'Chris', 'email': 'chris@nourishbalancethrive.com'}

In [50]:
def pydantic_to_mongoengine(pydantic_model):
    fields = {}
    for name, field in pydantic_model.__fields__.items():
        # This is a basic example and only handles str type; you would expand this for other types.
        if field.outer_type_ == str:
            fields[name] = StringField(required=field.required)
        # Add other field type conversions as needed
    return type(pydantic_model.__name__, (Document,), fields)


In [52]:
User = pydantic_to_mongoengine(UserParser)

In [54]:
User.__dict__

mappingproxy({'name': <mongoengine.fields.StringField at 0x146da2190>,
              'email': <mongoengine.fields.StringField at 0x1477e3650>,
              '_meta': {'abstract': False,
               'max_documents': None,
               'max_size': None,
               'ordering': [],
               'indexes': [],
               'id_field': 'id',
               'index_background': False,
               'index_opts': None,
               'delete_rules': None,
               'allow_inheritance': None,
               'collection': 'user_parser',
               'index_specs': []},
              '_is_base_cls': False,
              '_is_document': True,
              '_cached_reference_fields': [],
              '_fields': {'name': <mongoengine.fields.StringField at 0x146da2190>,
               'email': <mongoengine.fields.StringField at 0x1477e3650>,
               'id': <mongoengine.base.fields.ObjectIdField at 0x147674a10>},
              '_db_field_map': {'name': 'name', 'email': 'ema

# New

In [5]:
memory = X.load_memory()

load_memory filename='/Users/cck197/Downloads/virtual_greg/memory.pkl'


In [30]:
get_parsed_output("summarise the last sleep diary entry", memory_, SleepDiaryEntry)

SleepDiaryEntry(date=datetime.datetime(2023, 10, 5, 0, 0), in_bed=datetime.datetime(2023, 10, 12, 21, 0), tried_to_fall_asleep=datetime.datetime(2023, 10, 12, 23, 0), time_to_fall_asleep=30, times_awake=5, time_awake=50, final_wake_up=datetime.datetime(2023, 10, 12, 6, 30), out_of_bed=datetime.datetime(2023, 10, 12, 6, 30), sleep_quality='okay', medications='melatonin', notes='Slept in the van')

## Working

Answer the user query.
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"date": {"title": "Date", "description": "date of entry", "type": "string"}, "in_bed": {"title": "In Bed", "description": "time you went to bed", "type": "string"}, "tried_to_fall_asleep": {"title": "Tried To Fall Asleep", "description": "time you tried to fall asleep", "type": "string"}, "time_to_fall_asleep": {"title": "Time To Fall Asleep", "description": "how long it took you to fall asleep", "type": "integer"}, "times_awake": {"title": "Times Awake", "description": "how many times you woke up during the night", "type": "integer"}, "time_awake": {"title": "Time Awake", "description": "total time you were awake during the night", "type": "integer"}, "final_wake_up": {"title": "Final Wake Up", "description": "final wake up time", "type": "string"}, "out_of_bed": {"title": "Out Of Bed", "description": "time you got out of bed", "type": "string"}, "sleep_quality": {"title": "Sleep Quality", "description": "sleep quality", "type": "string"}, "medications": {"title": "Medications", "description": "medications or aids you used to help you sleep", "type": "string"}, "notes": {"title": "Notes", "description": "any other notes you'd like to add", "type": "string"}}, "required": ["date", "in_bed", "tried_to_fall_asleep", "time_to_fall_asleep", "times_awake", "time_awake", "final_wake_up", "out_of_bed", "sleep_quality", "medications", "notes"]}
```
summarise the last sleep diary entry
Previous conversation:[HumanMessage(content='hey'), AIMessage(content="Hello! I'm here to help you record your sleep diary. Before we begin, may I know your name, please?"), HumanMessage(content='Chris'), AIMessage(content='Nice to meet you, Chris! I also need your email address for our records. Could you please provide that?'), HumanMessage(content='chris@nourishbalancethrive.com'), AIMessage(content='Thank you, Chris. Now, is it a good time for you to record a sleep diary entry?'), HumanMessage(content='Oct 7'), AIMessage(content="Great, we'll record the sleep diary entry for October 7th. Let's start with the first question. What time did you go to bed on October 7th?"), HumanMessage(content='21:00'), AIMessage(content='Thank you, Chris. Now, what time did you try to fall asleep?'), HumanMessage(content='21:00'), AIMessage(content='Alright. How long did it take you to fall asleep, in minutes?'), HumanMessage(content='5'), AIMessage(content='Got it. How many times did you wake up during the night?'), HumanMessage(content='1'), AIMessage(content='I see. And how long were you awake during the night, in minutes?'), HumanMessage(content='5'), AIMessage(content='Noted. What was your final wake up time?'), HumanMessage(content='6:30'), AIMessage(content='Thank you. And what time did you get out of bed?'), HumanMessage(content='6:30'), AIMessage(content='Alright. How would you rate your sleep quality? You can choose from "very good", "good", "okay", "bad", "very bad".'), HumanMessage(content='okay'), AIMessage(content='Okay. Did you use any medications or aids to help you sleep?'), HumanMessage(content='melatonin, and for the notes I slept in the van 3'), AIMessage(content="Thank you for sharing that, Chris. Let's summarize:\n\n- Date of entry: October 7th\n- Time you went to bed: 21:00\n- Time you tried to fall asleep: 21:00\n- Time it took to fall asleep: 5 minutes\n- Number of times you woke up during the night: 1\n- Total time you were awake during the night: 5 minutes\n- Final wake up time: 6:30\n- Time you got out of bed: 6:30\n- Sleep quality: Okay\n- Sleep aids: Melatonin\n- Notes: Slept in the van 3\n\nYour sleep efficiency, which is the percentage of time spent asleep while in bed, is 96.7%. Anything over 85% is considered normal.\n\nIs this information correct?"), HumanMessage(content='yep!'), AIMessage(content='Great! Thank you for confirming, Chris. Your sleep diary entry has been successfully recorded. If you have any other questions or need further assistance, feel free to ask. 🚀🚀🚀'), HumanMessage(content='hey'), AIMessage(content='Hello again, Chris! How are you feeling today? Is it a good time to record another sleep diary entry?'), HumanMessage(content='chris@nourishbalancethrive.com'), AIMessage(content='Thank you, Chris. Now, is it a good time for you to record a sleep diary entry?'), HumanMessage(content='yes'), AIMessage(content="Great! Let's record the sleep diary entry for October 10th. What time did you go to bed?"), HumanMessage(content='actually no Oct 1'), AIMessage(content="Alright, we'll record the sleep diary entry for October 1st. What time did you go to bed on that day?"), HumanMessage(content='21:00'), AIMessage(content='Thank you, Chris. Now, what time did you try to fall asleep?'), HumanMessage(content='120'), AIMessage(content="I'm sorry, but the time you provided seems to be incorrect. Could you please provide the time in a 24-hour format? For example, if it's 1:20 AM, you should enter it as 01:20."), HumanMessage(content='23:00'), AIMessage(content='Got it. How long did it take you to fall asleep, in minutes?'), HumanMessage(content='30'), AIMessage(content='Noted. How many times did you wake up during the night?'), HumanMessage(content='5'), AIMessage(content='I see. And how long were you awake during the night, in minutes?'), HumanMessage(content='50'), AIMessage(content='Thank you for that information. What was your final wake up time?'), HumanMessage(content='6:30'), AIMessage(content='Thank you. And what time did you get out of bed?'), HumanMessage(content='6:30'), AIMessage(content='Alright. How would you rate your sleep quality? You can choose from "very good", "good", "okay", "bad", "very bad".'), HumanMessage(content='okay'), AIMessage(content='Okay. Did you use any medications or aids to help you sleep?'), HumanMessage(content='melatonin, and for the notes I slept in the van 4'), AIMessage(content="Thank you for sharing that, Chris. Let's summarize:\n\n- Date of entry: October 1st\n- Time you went to bed: 21:00\n- Time you tried to fall asleep: 23:00\n- Time it took to fall asleep: 30 minutes\n- Number of times you woke up during the night: 5\n- Total time you were awake during the night: 50 minutes\n- Final wake up time: 6:30\n- Time you got out of bed: 6:30\n- Sleep quality: Okay\n- Sleep aids: Melatonin\n- Notes: Slept in the van 4\n\nYour sleep efficiency, which is the percentage of time spent asleep while in bed, is 85.4%. Anything over 85% is considered normal.\n\nIs this information correct?"), HumanMessage(content='yep!'), AIMessage(content='Great! Thank you for confirming, Chris. Your sleep diary entry has been successfully recorded. If you have any other questions or need further assistance, feel free to ask. 🚀🚀🚀')]

{"date": "October 1st", "in_bed": "21:00", "tried_to_fall_asleep": "23:00", "time_to_fall_asleep": 30, "times_awake": 5, "time_awake": 50, "final_wake_up": "6:30", "out_of_bed": "6:30", "sleep_quality": "okay", "medications": "melatonin", "notes": "Slept in the van 4"}

Not working

Answer the user query.
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"date": {"title": "Date", "description": "date of entry", "type": "string"}, "in_bed": {"title": "In Bed", "description": "time you went to bed", "type": "string"}, "tried_to_fall_asleep": {"title": "Tried To Fall Asleep", "description": "time you tried to fall asleep", "type": "string"}, "time_to_fall_asleep": {"title": "Time To Fall Asleep", "description": "how long it took you to fall asleep", "type": "integer"}, "times_awake": {"title": "Times Awake", "description": "how many times you woke up during the night", "type": "integer"}, "time_awake": {"title": "Time Awake", "description": "total time you were awake during the night", "type": "integer"}, "final_wake_up": {"title": "Final Wake Up", "description": "final wake up time", "type": "string"}, "out_of_bed": {"title": "Out Of Bed", "description": "time you got out of bed", "type": "string"}, "sleep_quality": {"title": "Sleep Quality", "description": "sleep quality", "type": "string"}, "medications": {"title": "Medications", "description": "medications or aids you used to help you sleep", "type": "string"}, "notes": {"title": "Notes", "description": "any other notes you'd like to add", "type": "string"}}, "required": ["date", "in_bed", "tried_to_fall_asleep", "time_to_fall_asleep", "times_awake", "time_awake", "final_wake_up", "out_of_bed", "sleep_quality", "medications", "notes"]}
```
summarise the last sleep diary entry
Previous conversation:[AIMessage(content="what's your email?"), HumanMessage(content='chris@nourishbalancethrive.com'), HumanMessage(content='okay'), AIMessage(content='I see. How would you rate your sleep quality on a scale of "very good", "good", "okay", "bad", "very bad"?'), AIMessage(content="what's your name?"), HumanMessage(content='Chris'), AIMessage(content="what's your email?"), HumanMessage(content='chris@nourishbalancethrive.com'), HumanMessage(content='melatonin, and for the notes I slept in the van 5'), AIMessage(content='Thanks for sharing that, Chris. Here\'s the summary of your sleep diary entry for October 4, 2023:\n\n- You got in bed at 21:00.\n- You tried to fall asleep at 23:00.\n- It took you 30 minutes to fall asleep.\n- You woke up 5 times during the night.\n- You were awake for a total of 50 minutes during the night.\n- You finally woke up at 06:30.\n- You got out of bed at 06:30.\n- You rated your sleep quality as "okay".\n- You took melatonin as medication.\n- You noted that you "Slept in the van 5".\n\nYour sleep efficiency was 64.91%. Remember, anything over 85% is considered normal. Is this information correct? 🚀🚀🚀'), AIMessage(content="what's your name?"), HumanMessage(content='Chris'), AIMessage(content="what's your email?"), HumanMessage(content='chris@nourishbalancethrive.com'), HumanMessage(content='yep!'), AIMessage(content="Great! I'm glad we could record your sleep diary entry for October 4, 2023. Remember, keeping a regular sleep diary can help identify patterns and areas for improvement in your sleep habits. Keep up the good work! 🚀🚀🚀"), AIMessage(content="what's your name?"), HumanMessage(content='Chris'), AIMessage(content="what's your email?"), HumanMessage(content='chris@nourishbalancethrive.com'), HumanMessage(content='save that entry to the database'), AIMessage(content="I'm sorry, but as an AI, I don't have the ability to directly interact with databases. However, I can help you format your data in a way that's ready to be input into a database. Here's your sleep diary entry in a database-friendly format:\n\n- Date: October 4, 2023\n- Bedtime: 21:00\n- Sleep Attempt Time: 23:00\n- Time to Fall Asleep: 30 minutes\n- Number of Awakenings: 5\n- Total Awake Time: 50 minutes\n- Final Wake Time: 06:30\n- Out of Bed Time: 06:30\n- Sleep Quality: Okay\n- Medications: Melatonin\n- Notes: Slept in the van 5\n\nPlease let me know if you need help with anything else. 🚀🚀🚀"), AIMessage(content="what's your name?"), HumanMessage(content='Chris'), AIMessage(content="what's your email?"), HumanMessage(content='chris@nourishbalancethrive.com'), HumanMessage(content='hey'), AIMessage(content="Hello Chris, nice to meet you. I'm your AI clinician. I'm going to help you with your sleep diary. If you have any sleep diary entries, please share them with me."), AIMessage(content="what's your name?"), HumanMessage(content='Chris'), AIMessage(content="what's your email?"), HumanMessage(content='chris@nourishbalancethrive.com'), HumanMessage(content="what's my email?"), AIMessage(content="Your email is chris@nourishbalancethrive.com. Now, let's move on to the sleep diary. Is now a good time to record a diary entry?"), AIMessage(content="what's your name?"), HumanMessage(content='Chris'), AIMessage(content="what's your email?"), HumanMessage(content='chris@nourishbalancethrive.com'), HumanMessage(content='Oct 5'), AIMessage(content="I'm sorry, but I couldn't find a sleep diary entry for October 5, 2023. Would you like to create a new entry for this date?"), AIMessage(content="what's your name?"), HumanMessage(content='Chris'), AIMessage(content="what's your email?"), HumanMessage(content='chris@nourishbalancethrive.com'), HumanMessage(content='yep'), AIMessage(content="Great! Let's start with the time you went to bed on October 5, 2023."), AIMessage(content="what's your name?"), HumanMessage(content='Chris'), AIMessage(content="what's your email?"), HumanMessage(content='chris@nourishbalancethrive.com'), HumanMessage(content='21:00'), AIMessage(content='Thanks, Chris. Now, what time did you try to fall asleep?'), AIMessage(content="what's your name?"), HumanMessage(content='Chris'), AIMessage(content="what's your email?"), HumanMessage(content='chris@nourishbalancethrive.com'), HumanMessage(content='23:00'), AIMessage(content='Got it. How long did it take you to fall asleep?'), AIMessage(content="what's your name?"), HumanMessage(content='Chris'), AIMessage(content="what's your email?"), HumanMessage(content='chris@nourishbalancethrive.com'), HumanMessage(content='30'), AIMessage(content='Thanks for sharing that. How many times did you wake up during the night?'), AIMessage(content="what's your name?"), HumanMessage(content='Chris'), AIMessage(content="what's your email?"), HumanMessage(content='chris@nourishbalancethrive.com'), HumanMessage(content='5'), AIMessage(content='I see. And how long were you awake during these times? Please provide the total time in minutes.'), AIMessage(content="what's your name?"), HumanMessage(content='Chris'), AIMessage(content="what's your email?"), HumanMessage(content='chris@nourishbalancethrive.com'), HumanMessage(content='50'), AIMessage(content='Thanks for that information. What time did you finally wake up?'), AIMessage(content="what's your name?"), HumanMessage(content='Chris'), AIMessage(content="what's your email?"), HumanMessage(content='chris@nourishbalancethrive.com'), HumanMessage(content='6:30'), AIMessage(content='Thanks for sharing that. And what time did you get out of bed?'), AIMessage(content="what's your name?"), HumanMessage(content='Chris'), AIMessage(content="what's your email?"), HumanMessage(content='chris@nourishbalancethrive.com'), HumanMessage(content='okay'), AIMessage(content='I see. How would you rate your sleep quality on a scale of "very good", "good", "okay", "bad", "very bad"?'), AIMessage(content="what's your name?"), HumanMessage(content='Chris'), AIMessage(content="what's your email?"), HumanMessage(content='chris@nourishbalancethrive.com'), HumanMessage(content='melatonin, and for the notes I slept in the van'), AIMessage(content='Thanks for sharing that, Chris. Here\'s the summary of your sleep diary entry for October 5, 2023:\n\n- You got in bed at 21:00.\n- You tried to fall asleep at 23:00.\n- It took you 30 minutes to fall asleep.\n- You woke up 5 times during the night.\n- You were awake for a total of 50 minutes during the night.\n- You finally woke up at 06:30.\n- You got out of bed at 06:30.\n- You rated your sleep quality as "okay".\n- You took melatonin as medication.\n- You noted that you "Slept in the van".\n\nYour sleep efficiency was 64.91%. Remember, anything over 85% is considered normal. Is this information correct?'), AIMessage(content="what's your name?"), HumanMessage(content='Chris'), AIMessage(content="what's your email?"), HumanMessage(content='chris@nourishbalancethrive.com'), HumanMessage(content='yep!'), AIMessage(content="Great! I've saved your sleep diary entry for October 5, 2023. Remember, keeping a regular sleep diary can help identify patterns and areas for improvement in your sleep habits. Keep up the good work! 🚀🚀🚀")]

{"properties": {"date": {"title": "Date", "description": "date of entry", "type": "string"}, "in_bed": {"title": "In Bed", "description": "time you went to bed", "type": "string"}, "tried_to_fall_asleep": {"title": "Tried To Fall Asleep", "description": "time you tried to fall asleep", "type": "string"}, "time_to_fall_asleep": {"title": "Time To Fall Asleep", "description": "how long it took you to fall asleep", "type": "integer"}, "times_awake": {"title": "Times Awake", "description": "how many times you woke up during the night", "type": "integer"}, "time_awake": {"title": "Time Awake", "description": "total time you were awake during the night", "type": "integer"}, "final_wake_up": {"title": "Final Wake Up", "description": "final wake up time", "type": "string"}, "out_of_bed": {"title": "Out Of Bed", "description": "time you got out of bed

In [5]:
user = get_user(memory); user

User(name='Chris', email='chris@nourishbalancethrive.com')

In [3]:
entry = get_sleep_diary_entry(memory); entry

OutputParserException: Failed to parse SleepDiaryEntry from completion 

The output should be formatted as a JSON instance that conforms to the following schema:

{"properties": {"date": {"title": "Date", "description": "date of entry", "type": "string"}, "in_bed": {"title": "In Bed", "description": "time you went to bed", "type": "string"}, "tried_to_fall_asleep": {"title": "Tried To Fall Asleep", "description": "time you tried to fall asleep", "type": "string"}, "time_to_fall_asleep": {"title": "Time To Fall Asleep", "description": "how long it took you to fall asleep", "type": "integer"}, "times_awake": {"title": "Times Awake", "description": "how many times you woke up during the night", "type": "integer"}, "time_awake": {"title": "Time Awake", "description": "total time you were awake during the night", "type": "integer"}, "final_wake_up": {"title": "Final Wake Up", "description": "final wake up time", "type": "string"}, "out_of_bed. Got: Expecting ',' delimiter: line 1 column 776

In [5]:
db_entry = save_sleep_diary_entry(db_user, entry)

In [6]:
db_entry.to_mongo().to_dict()

{'_id': ObjectId('652738d055cc9d6f13c11cf5'),
 'user': ObjectId('6526d86e6015be5b3434ac13'),
 'date': datetime.datetime(2023, 10, 1, 0, 0),
 'in_bed': datetime.datetime(2023, 10, 1, 21, 0),
 'tried_to_fall_asleep': datetime.datetime(2023, 10, 1, 23, 0),
 'time_to_fall_asleep': 30,
 'times_awake': 5,
 'time_awake': 50,
 'final_wake_up': datetime.datetime(2023, 10, 2, 6, 30),
 'out_of_bed': datetime.datetime(2023, 10, 2, 6, 30),
 'sleep_quality': 'okay',
 'medications': 'melatonin',
 'notes': 'Slept in the van 4'}

In [21]:
DBSleepDiaryEntry.objects.delete()

7

In [44]:
DBSleepDiaryEntry.objects().order_by('-id').first().to_mongo().to_dict()

{'_id': ObjectId('65270f56f2031c8434501e59'),
 'user': ObjectId('6526d86e6015be5b3434ac13'),
 'date': datetime.datetime(2023, 10, 11, 0, 0),
 'in_bed': datetime.datetime(2023, 10, 11, 21, 15),
 'tried_to_fall_asleep': datetime.datetime(2023, 10, 11, 21, 15),
 'time_to_fall_asleep': 5,
 'times_awake': 1,
 'time_awake': 5,
 'final_wake_up': datetime.datetime(2023, 10, 12, 6, 15),
 'out_of_bed': datetime.datetime(2023, 10, 12, 6, 15),
 'sleep_quality': 'Good',
 'medications': 'Melatonin',
 'notes': 'Slept in the van, it was raining'}

In [18]:
DBUser.objects.delete()

1

In [13]:
date_fields

['date', 'in_bed', 'tried_to_fall_asleep', 'final_wake_up', 'out_of_bed']

In [8]:
sleep_data = entry.dict()

In [9]:
adjust_to_baseline_date(sleep_data)

{'date': datetime.datetime(2023, 10, 1, 0, 0),
 'in_bed': datetime.datetime(2023, 10, 1, 21, 0),
 'tried_to_fall_asleep': datetime.datetime(2023, 10, 1, 23, 0),
 'time_to_fall_asleep': 30,
 'times_awake': 5,
 'time_awake': 50,
 'final_wake_up': datetime.datetime(2023, 10, 1, 6, 30),
 'out_of_bed': datetime.datetime(2023, 10, 1, 6, 30),
 'sleep_quality': 'okay',
 'medications': 'melatonin',
 'notes': 'Slept in the van 4'}

In [14]:
normalized_data = make_monotonic(sleep_data); normalized_data

in_bed
tried_to_fall_asleep
final_wake_up
out_of_bed


{'date': datetime.datetime(2023, 10, 7, 0, 0),
 'in_bed': datetime.datetime(2023, 10, 11, 21, 15),
 'tried_to_fall_asleep': datetime.datetime(2023, 10, 12, 21, 10),
 'final_wake_up': datetime.datetime(2023, 10, 12, 23, 15),
 'out_of_bed': datetime.datetime(2023, 10, 13, 6, 15),
 'sleep_quality': 'Good',
 'medications': 'Melatonin',
 'notes': 'Slept in the van, it was raining'}

In [16]:
date = date_parser("2023-09-01")

In [22]:
DBSleepDiaryEntry.objects(user=db_user, date=date)

[<SleepDiaryEntry: SleepDiaryEntry object>]

In [5]:
[e.date for e in DBSleepDiaryEntry.objects()]

[datetime.datetime(2023, 9, 1, 0, 0),
 datetime.datetime(2023, 9, 2, 0, 0),
 datetime.datetime(2023, 9, 3, 0, 0),
 datetime.datetime(2023, 9, 4, 0, 0),
 datetime.datetime(2023, 9, 5, 0, 0)]

In [6]:
DBSleepDiaryEntry.objects(user=db_user, date=date_parser("2023-10-01")).first().to_mongo()

SON([('_id', ObjectId('652738d055cc9d6f13c11cf5')), ('user', ObjectId('6526d86e6015be5b3434ac13')), ('date', datetime.datetime(2023, 10, 1, 0, 0)), ('in_bed', datetime.datetime(2023, 10, 1, 21, 0)), ('tried_to_fall_asleep', datetime.datetime(2023, 10, 1, 23, 0)), ('time_to_fall_asleep', 30), ('times_awake', 5), ('time_awake', 50), ('final_wake_up', datetime.datetime(2023, 10, 2, 6, 30)), ('out_of_bed', datetime.datetime(2023, 10, 2, 6, 30)), ('sleep_quality', 'okay'), ('medications', 'melatonin'), ('notes', 'Slept in the van 4')])

In [2]:
DBISIEntry.objects()

[<ISIEntry: ISIEntry object>]